# Computation of magnification factor using pycortex
__To do :__
- [x] adapt it to vertice analysis

In [3]:
# Stop warnings
import warnings
warnings.filterwarnings("ignore")

# General imports
import cortex
import importlib
import json
import numpy as np
import os
import sys
sys.path.append("{}/../../../utils".format(os.getcwd()))
from pycortex_utils import draw_cortex, set_pycortex_config_file, load_surface_pycortex, get_rois
import nibabel as nb

# Define analysis parameters
with open('../../../settings.json') as f:
    json_s = f.read()
    analysis_info = json.loads(json_s)
tasks = analysis_info["task_names"]
task = tasks[2]
rois = analysis_info["rois"]
vert_dist_th = analysis_info['vertex_pcm_rad']
formats = analysis_info['formats']

# debug
formats = ['fsnative','170k']
rois = ['V1']

# Inputs
# main_dir = '/home/mszinte/disks/meso_S/data'
# main_dir = '/Users/uriel/disks/meso_shared/'
main_dir = '/home/ulascombes/disks/meso_shared/'
project_dir = 'RetinoMaps'
subject = 'sub-01'
deriv_fn_label = 'avg-gridfit'
model = 'gauss'

# Set pycortex db and colormaps
cortex_dir = "{}/{}/derivatives/pp_data/cortex".format(main_dir, project_dir)
set_pycortex_config_file(cortex_dir)
importlib.reload(cortex)


for format_, pycortex_subject in zip(formats, [subject, 'sub-170k']):
    # define directories and fn
    prf_dir = "{}/{}/derivatives/pp_data/{}/{}/prf".format(main_dir, project_dir, 
                                                           subject, format_)
    fit_dir = "{}/fit".format(prf_dir)
    prf_deriv_dir = "{}/prf_derivatives".format(prf_dir)

    if format_ == 'fsnative':
        atlas_name = None
        surf_size = None
        deriv_avg_fn_L = '{}/{}_task-{}_hemi-L_fmriprep_dct_avg_prf-deriv_{}_gridfit.func.gii'.format(
            prf_deriv_dir, subject, task, model)
        deriv_avg_fn_R = '{}/{}_task-{}_hemi-R_fmriprep_dct_avg_prf-deriv_{}_gridfit.func.gii'.format(
            prf_deriv_dir, subject, task, model)
        
        results = load_surface_pycortex(L_fn=deriv_avg_fn_L, 
                                        R_fn=deriv_avg_fn_R, 
                                        return_img=True)

        deriv_mat, img_L, img_R = results['data_concat'], results['img_L'], results['img_R']


        
    elif format_ == '170k':
        deriv_avg_fn = '{}/{}_task-{}_fmriprep_dct_avg_prf-deriv_{}_gridfit.dtseries.nii'.format(
            prf_deriv_dir, subject, task, model)
        atlas_name = 'mmp'
        surf_size = '59k'
        results = load_surface_pycortex(brain_fn=deriv_avg_fn,
                                        return_img=True,
                                        return_59k_mask=True, 
                                        return_170k_mask=True, 
                                        return_source_data=True)
        deriv_mat, mask_59k, mask_170k, deriv_mat_170k, img  = results['data_concat'], results['mask_59k'], results['mask_170k'], results['source_data'], results['img']



In [4]:
# get surfaces for each hemisphere
surfs = [cortex.polyutils.Surface(*d) for d in cortex.db.get_surf(pycortex_subject, "flat")]
surf_lh, surf_rh = surfs[0], surfs[1]
# get the vertices number per hemisphere
lh_vert_num, rh_vert_num = surf_lh.pts.shape[0], surf_rh.pts.shape[0]
vert_num = lh_vert_num + rh_vert_num

# get a dicst with the surface vertices contained in each ROI
# roi_verts_dict = cortex.utils.get_roi_verts(subject, mask=False)
roi_verts_dict = get_rois(pycortex_subject, 
                          return_concat_hemis=True, 
                          rois=rois, 
                          mask=False, 
                          atlas_name=atlas_name, 
                          surf_size=surf_size)
#### TO REPLACE BY YOUR NEW FUNCTION TO GET ROIS FROM NPZ IN CASE OF SUB-17K

# derivatives settings
rsq_idx, ecc_idx, polar_real_idx, polar_imag_idx , size_idx, \
    amp_idx, baseline_idx, x_idx, y_idx, hrf_1_idx, hrf_2_idx = \
    0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10
if model == 'gauss':
    loo_rsq_idx = 11
elif model == 'dn':
    srf_amp_idx = 11
    sff_size = 12
    neural_baseline_idx = 13
    surround_baseline = 14
    loo_rsq_idx = 15
elif model == 'css':
    n_idx = 11
    loo_rsq_idx = 12

# parameters
vert_rsq_data = deriv_mat[rsq_idx, ...]
vert_x_data = deriv_mat[x_idx, ...]
vert_y_data = deriv_mat[y_idx, ...]
vert_size_data = deriv_mat[size_idx, ...]
vert_ecc_data = deriv_mat[ecc_idx, ...]

# create empty results
vert_cm = np.zeros(vert_num)*np.nan

In [18]:
mask_170k.shape

(170494,)

In [5]:
for roi in rois:
    # find ROI vertex
    roi_vert_lh_idx = roi_verts_dict[roi][roi_verts_dict[roi]<lh_vert_num]
    roi_vert_rh_idx = roi_verts_dict[roi][roi_verts_dict[roi]>=lh_vert_num]
    roi_surf_lh_idx = roi_vert_lh_idx
    roi_surf_rh_idx = roi_vert_rh_idx-lh_vert_num

    # get mean distance of surounding vertices included in threshold
    vert_lh_rsq, vert_lh_size = vert_rsq_data[:lh_vert_num], vert_size_data[:lh_vert_num]
    vert_lh_x, vert_lh_y = vert_x_data[:lh_vert_num], vert_y_data[:lh_vert_num]
    vert_rh_rsq, vert_rh_size = vert_rsq_data[lh_vert_num:], vert_size_data[lh_vert_num:]
    vert_rh_x, vert_rh_y = vert_x_data[lh_vert_num:], vert_y_data[lh_vert_num:]

    for hemi in ['lh','rh']:
        if hemi == 'lh':
            surf = surf_lh
            roi_vert_idx, roi_surf_idx = roi_vert_lh_idx, roi_surf_lh_idx
            vert_rsq, vert_x, vert_y, vert_size = vert_lh_rsq, vert_lh_x, vert_lh_y, vert_lh_size
        elif hemi == 'rh':
            surf = surf_rh
            roi_vert_idx, roi_surf_idx = roi_vert_rh_idx, roi_surf_rh_idx
            vert_rsq, vert_x, vert_y, vert_size = vert_rh_rsq, vert_rh_x, vert_rh_y, vert_rh_size

        desc = 'ROI -> {} / Hemisphere -> {}'.format(roi, hemi)
        print(desc)
        for i, (vert_idx, surf_idx) in enumerate(zip(roi_vert_idx, roi_surf_idx)):

            if vert_rsq[surf_idx] > 0:

                # get geodesic distances (mm)
                try :
                    geo_patch = surf.get_geodesic_patch(radius=vert_dist_th, vertex=surf_idx)
                except Exception as e:
                    print("Vertex #{}: error: {} within {} mm".format(vert_idx, e, vert_dist_th))
                    geo_patch['vertex_mask'] = np.zeros(surf.pts.shape[0]).astype(bool)
                    geo_patch['geodesic_distance'] = []

                vert_dist_th_idx  = geo_patch['vertex_mask']
                vert_dist_th_dist = np.ones_like(vert_dist_th_idx)*np.nan
                vert_dist_th_dist[vert_dist_th_idx] = geo_patch['geodesic_distance']

                # exclude vextex out of roi
                vert_dist_th_not_in_roi_idx = [idx for idx in np.where(vert_dist_th_idx)[0] if idx not in roi_surf_idx]
                vert_dist_th_idx[vert_dist_th_not_in_roi_idx] = False
                vert_dist_th_dist[vert_dist_th_not_in_roi_idx] = np.nan

                if np.sum(vert_dist_th_idx) > 0:

                    # compute average geodesic distance excluding distance to itself (see [1:])
                    vert_geo_dist_avg = np.nanmean(vert_dist_th_dist[1:])

                    # get prf parameters of vertices in geodesic distance threshold
                    vert_ctr_x, vert_ctr_y = vert_x[surf_idx], vert_y[surf_idx]
                    vert_dist_th_idx[surf_idx] = False
                    vert_srd_x, vert_srd_y = np.nanmean(vert_x[vert_dist_th_idx]), np.nanmean(vert_y[vert_dist_th_idx])

                    # compute prf center suround distance (deg)
                    vert_prf_dist = np.sqrt((vert_ctr_x - vert_srd_x)**2 + (vert_ctr_y - vert_srd_y)**2)

                    # compute cortical magnification in mm/deg (surface distance / pRF positon distance)
                    vert_cm[vert_idx] = vert_geo_dist_avg/vert_prf_dist


ROI -> V1 / Hemisphere -> lh
Vertex #42060: error: could not find suitable radius within 2 mm
Vertex #42080: error: could not find suitable radius within 2 mm
Vertex #43185: error: could not find suitable radius within 2 mm
Vertex #43295: error: could not find suitable radius within 2 mm
Vertex #43353: error: could not find suitable radius within 2 mm
Vertex #43355: error: could not find suitable radius within 2 mm
Vertex #43367: error: could not find suitable radius within 2 mm
Vertex #43411: error: could not find suitable radius within 2 mm
Vertex #43463: error: could not find suitable radius within 2 mm
Vertex #43468: error: could not find suitable radius within 2 mm
Vertex #43478: error: could not find suitable radius within 2 mm
Vertex #43525: error: could not find suitable radius within 2 mm
Vertex #43534: error: could not find suitable radius within 2 mm
Vertex #43579: error: could not find suitable radius within 2 mm
Vertex #43582: error: could not find suitable radius within 2

In [9]:
deriv_mat_new = np.zeros((deriv_mat.shape[0]+1, deriv_mat.shape[1]))*np.nan
deriv_mat_new[0:-1,...] = deriv_mat
deriv_mat_new[-1,...] = vert_cm

In [23]:
def make_image_pycortex(data, 
                        maps_names=None,
                        img_L=None, 
                        img_R=None, 
                        lh_vert_num=None, 
                        rh_vert_num=None, 
                        img=None, 
                        brain_mask_59k=None, 
                        brain_mask_170k=None):
    """
    Make a Cifti or Gifti image with data imported by PyCortex. This means that Gifti data 
    will be split by hemisphere, and Cifti data will be transformed back into 170k size.

    Parameters:
    - data: numpy array, your data.
    - maps_names: list of strings, optional, names for the mapped data.
    - img_L: Gifti Surface, left hemisphere surface object.
    - img_R: Gifti Surface, right hemisphere surface object.
    - lh_vert_num: int, number of vertices in the left hemisphere.
    - rh_vert_num: int, number of vertices in the right hemisphere.
    - img: Cifti Volume, source volume for mapping onto the surface.
    - brain_mask_59k: numpy array, optional, brain mask for 59k vertices (output of the from_170k_to_59k function).
    - brain_mask_170k: numpy array, optional, brain mask for 170k vertices (output of the from_170k_to_59k function).

    Returns:
    If mapping onto separate hemispheres (img_L and img_R provided):
    - new_img_L: Gifti img, new surface representing data on the left hemisphere.
    - new_img_R: Gifti img, new surface representing data on the right hemisphere.

    If mapping onto a single hemisphere (img provided):
    - new_img: Cifti img, new surface representing data on 170k size.
    """
    from cifti_utils import from_59k_to_170k
    from surface_utils import make_surface_image 
    import numpy as np
    

    if img_L and img_R: 
        data_L = data[:,:lh_vert_num]
        data_R = data[:,-rh_vert_num:]

        new_img_L = make_surface_image(data_L, img_L, maps_names=maps_names)
        new_img_R = make_surface_image(data_R, img_R, maps_names=maps_names)
        return new_img_L, new_img_R
        
    elif img:
        data_170k = from_59k_to_170k(data_59k=data, 
                                     brain_mask_59k=brain_mask_59k, 
                                     brain_mask_170k=brain_mask_170k)
        
        new_img = make_surface_image(data=data, 
                                     source_img=img, 
                                     maps_names=maps_names)
        return new_img


In [24]:
def from_59k_to_170k(data_59k, brain_mask_59k, brain_mask_170k):
    """
    Transform 59k data into 170k data by filling non-59k vertices with numpy.nan.

    Parameters
    ----------
    
    data_59k : The 59k data you want to transform into 170k.
    data_170k_orig : The original 170k data from which your 59k data originated.
    brain_mask_59k : 59k brain mask output from from_170k_to_59k.
    
    Returns
    -------
    The transformed data in 170k format with non-59k vertices filled with numpy.nan.
    """
    import numpy as np
    
    # mask 59k data to optain only cortex vertex (and not medial wall vertices ) 
    data_59k = data_59k[:,brain_mask_59k]

    # create an 170k full nan array
    n_vertex_170k = brain_mask_170k.shape[0]
    n_TRs = data_59k.shape[0]
    data_170k_final = np.full((n_TRs,n_vertex_170k),np.nan)
    
    # fill the 170k array with the cortex data
    data_170k_final[:,brain_mask_170k] = data_59k
    
    return data_170k_final

In [25]:
new_img = save_surface_pycortex(data=deriv_mat_new, 
                          maps_names=None,
                          img_L=None, 
                          img_R=None, 
                          lh_vert_num=None, 
                          rh_vert_num=None, 
                          img=img, 
                          brain_mask_59k=mask_59k, 
                          brain_mask_170k=mask_170k)

# Brouillon

In [ ]:
def load_rois_atlas(atlas_name, surf_size, rois=None, mask=True, path_to_atlas=None):
    """
    Loads ROIs from an atlas.

    Parameters
    ----------
    atlas_name : str
        The name of the atlas.
    surf_size : str
        Size of the surface, either '59k' or '170k'.
    rois : list of str, optional
        List of ROIs you want to extract. If None, all ROIs are returned. 
        Default is None.
    mask : bool, optional
        If True, returns the ROI masks. If False, returns the indices where the masks are True.
        Default is True.
    path_to_atlas : str, optional
        Path to the directory containing the atlas data. If not provided, the function looks for the atlas 
        data in the default directory.

    Returns
    -------
    rois_masks : dict
        A dictionary where the keys represent the ROIs and the values correspond 
        to the respective masks for each hemisphere.

    Raises
    ------
    ValueError
        If 'surf_size' is not '59k' or '170k'.
    """
    import os
    import numpy as np
    
    # Validating surf_size
    if surf_size not in ['59k', '170k']:
        raise ValueError("Invalid value for 'surf_size'. It should be either '59k' or '170k'.")
        
    # Loading data from the specified path or default directory
    if path_to_atlas:
        data = np.load(path_to_atlas)
    else:    
        atlas_dir = os.path.abspath(os.path.join(os.getcwd(), "../../../atlas/"))
        filename = "{}_atlas_rois_{}.npz".format(atlas_name, surf_size)
        data = np.load('{}/{}'.format(atlas_dir, filename))
    
    rois_dict = dict(data)
    
    # Handling the case where mask is False
    if not mask:
        # Returning indices where the masks are True
        rois_dict = {roi: np.where(rois_dict[roi])[0] for roi in rois_dict}
        
    # Filtering ROIs if rois is provided
    if rois is None:
        return rois_dict
    elif isinstance(rois, list):
        filtered_rois = {roi: rois_dict[roi] for roi in rois if roi in rois_dict}
        return filtered_rois
    else:
        raise ValueError("Invalid value for 'rois'. It should be either None or a list of ROI names.")
        
def data_from_rois(fn, subject, rois, return_concat_hemis):
    """
    Load a surface, and returne vertex only data from the specified ROIs
    ----------
    fn : surface filename
    subject : subject 
    rois : list of rois you want extract 
    
    Returns
    -------
    img : the image load from fn   
    data_roi : numpy rois data 
              2 dim (time x vertices from all the rois)  
              
    roi_idx : indices of the rois vertices 
    
    
    data_hemi : numpy stacked data
                2 dim (time x vertices)    
    """
    import cortex
    from surface_utils import load_surface

    # import data 
    img, data = load_surface(fn=fn)
    len_data = data.shape[1]
    
    # get rois mask 
    if fn.endswith('.gii'):
        roi_verts = cortex.get_roi_verts(subject=subject, 
                                         roi= rois, 
                                         mask=True)
    elif fn.endswith('.nii'):
        if len_data > 60000:
            surf_size = '170k'
        else:
            surf_size = '59k'
            
        roi_verts = load_rois_atlas(atlas_name='mmp',surf_size=surf_size)
    
    na_vertices = np.isnan(data).any(axis=0)
    
    # create a brain mask  
    brain_mask = np.any(list(roi_verts.values()), axis=0)
    
    # create a hemi mask  
    if 'hemi-L' in fn:
        hemi_mask = brain_mask[:len_data]
        for i, na_vertices in enumerate(na_vertices):
            hemi_mask[i] = not na_vertices and hemi_mask[i]
        
    elif 'hemi-R' in fn: 
        hemi_mask = brain_mask[-len_data:]
        for i, na_vertices in enumerate(na_vertices):
            hemi_mask[i] = not na_vertices and hemi_mask[i]
    else: 
        for i, na_vertices in enumerate(na_vertices):
            brain_mask[i] = not na_vertices and brain_mask[i]
        
    roi_idx = np.where(hemi_mask)[0]
    
    data_roi = data[:,hemi_mask]

        
    return img, data, data_roi, roi_idx

def get_rois(subject, return_concat_hemis=False,rois=None, mask=True, atlas_name=None, surf_size=None):
    """
    Accesses single hemisphere ROI masks for GIFTI and atlas ROI for CIFTI.

    Parameters
    ----------
    subject : str
        Subject name in the pycortex database.
    return_concat_hemis : bool, optional
        Indicates whether to return concatenated hemisphere ROIs. Defaults to False.
    rois : list of str, optional
        List of ROIs you want to extract.
    mask : bool, optional
        Indicates whether to mask the ROIs. Defaults to True.
    atlas_name : str, optional
        If atlas_name is not None, subject has to be a template subject (i.e., sub-170k).
        If provided, `surf_size` must also be specified.
    surf_size : str, optional
        The size in which you want the ROIs. It should be '59k' or '170k'. 
        Required if `atlas_name` is provided.

    Returns
    -------
    rois_masks : dict
        A dictionary where the keys represent the ROIs and the values correspond to the respective masks for each hemisphere.
    """
    import cortex
    from surface_utils import load_surface
    
    surfs = [cortex.polyutils.Surface(*d) for d in cortex.db.get_surf(subject, "flat")]
    surf_lh, surf_rh = surfs[0], surfs[1]
    lh_vert_num, rh_vert_num = surf_lh.pts.shape[0], surf_rh.pts.shape[0]

    
    # get rois 
    if atlas_name :
        roi_verts = load_rois_atlas(atlas_name=atlas_name, 
                                    surf_size=surf_size,
                                    rois=rois, 
                                    mask=mask)
        return roi_verts

    else:
        roi_verts = cortex.get_roi_verts(subject=subject, 
                                         roi=rois, 
                                         mask=mask)
    
    rois_masks_L = {roi: data[:lh_vert_num] for roi, data in roi_verts.items()}
    rois_masks_R = {roi: data[-rh_vert_num:] for roi, data in roi_verts.items()}

    if return_concat_hemis :
        return roi_verts
    else:
        return rois_masks_L, rois_masks_R
